-This note book is the iterartion 04 of the HPCC_GNN_Project-

References:https://www.ece.ualberta.ca/~elliott/ee552/studentAppNotes/2003_w/misc/bmp_file_format/bmp_file_format.htm

1.Image type being processed is BMP

2.A manual function using struct.unpack is written which ensures a higher control over the data

3.The main aim is to extract the pixel data from the hexdata generated from spraying the data.



The MS-Windows standard format is BMP and was developed as a device-independent bitmap (DIB) format that will allow Windows to display the bitmap on any type of display device. The term "device independent" means that the bitmap specifies pixel color in a form independent of the method used by a display to represent color. This file format can be stored uncompressed, so reading BMP files is fairly simple; most other graphics formats are compressed, and some, like GIF, are difficult to decompress.

In [2]:
import io
import struct
from PIL import Image
import binascii
import numpy as np
from io import BytesIO


Since the blob spraying returns the hex data of an image , we are trying to simulate the same by using io and Pil library. (In this note book we have used a BMP format)

In [3]:
# img=Image.open("C:/Users/rohan/Pictures/Camera Roll/X13_wallpaper_final_16x9_FHD.jpg")
img=Image.open("C:/Users/rohan/Desktop/uktkjxhcb/sampleFormats/sample_1920x1280_BMP.bmp")
print(img)


<PIL.BmpImagePlugin.BmpImageFile image mode=RGB size=1920x1280 at 0x27D7CAF0D60>


Uing hexlify() imported from binascii library to obtain the hex data of the image.

In [4]:
# with open("C:/Users/rohan/Pictures/Camera Roll/X13_wallpaper_final_16x9_FHD.jpg", "rb") as image_file:
with open("C:/Users/rohan/Desktop/uktkjxhcb/sampleFormats/sample_1920x1280_BMP.bmp", "rb") as image_file:
    image_hex = binascii.hexlify(image_file.read())
    # print(image_hex)
    print(image_hex[:1500])
    

b'424d8a807000000000008a0000007c0000008007000000050000010018000000000000807000000000000000000000000000000000000000ff0000ff0000ff000000000000ff4247527380c2f52860b81e152085eb01403333138066662640666606a09999093c0ad703245c8f32000000000000000000000000040000000000000000000000000000000907070200000806060c0a0a0202020a0a0a0e0e0e0606060303030808080a08080301010400000904050b05060601020301010808080d0d0d0b0b0b0806060604040705050907070a0808090707060606040606040606020706030807040809000407060a0f090c10090d0e0e0d0f0a0a0a0806050b09080c08070c08070a08070a08070809070708060204040002020b080a0500011204080c0306060a0b000e0d0d2220101e1d09080a130c0f1d161b13121600040901070c130c130c060b09080a050a09070c0b090e0d0c110f0f1412141715161917191a181f201e2222221b1b1b1110120b0a0c0c080d0d090e0e0d110d0c10110d12120e13130e100f0a0c0d08090d08090805070000010c0a1006090e01030d00030c070d18050e181d27314b5a63566e7a1c3c470f3743022d36001f29042128001217030c0f0c080d130c0f1a0e14170d131209130e0a10080c0d030908020706060b0a0e0e0e0e0e0e0f0a0b0d080913

The hex data into binary data using unhexlify() function.

In [5]:
binary_data = binascii.unhexlify(image_hex)


struct.unpack returns a tuple, even if it contains only one value

And hence we need to compare it individually

The BMP format uses the two-byte sequence 0x42 0x4D

In [6]:


def is_bitmap(image_hex):
    bmp_signature = b'BM'
    unpacked_signature = struct.unpack("2B", binary_data[:2])
    return bytes([unpacked_signature[0]]) == bmp_signature[:1] and bytes([unpacked_signature[1]]) == bmp_signature[1:]

# Example usage

print(is_bitmap(image_hex))


True


Pixels are stored "upside-down" with respect to normal image raster scan order, starting in the lower left corner, going from left to right, and then row by row from the bottom to the top of the image.2 Uncompressed Windows bitmaps can also be stored from the top row to the bottom, if the image height value is negative.(https://grapherhelp.goldensoftware.com/subsys/subsys_bitmap_file_description.htm)

struct unpack is used to extract these header in the form of "<IHHIIIIIIII "

In [12]:
def extract_pixel_data(image_hex):
    # Unpack the header information from the binary data
    header = struct.unpack("<IHHIIIIIIII", image_hex[:40])
    image_width = header[6]
    image_height = header[7]
    pixel_offset = header[10]
# Below we are calculating the pixel data size
# Since the BMP image must be a multiple of 4 so it will be just rounded to the nearest 4's multiple (https://stackoverflow.com/questions/15313792/what-should-we-do-with-a-bitmap-file-when-its-rowsize-isnt-multiple-of-4)
    row_size = (image_width * 3 + 3) & ~3
    image_size = row_size * abs(image_height)


    pixel_data = image_hex[pixel_offset:pixel_offset + image_size]

    return pixel_data
pixel_data = extract_pixel_data(binary_data)
# printing the first 1000 indexes of  pixel data for verification purposes
print(pixel_data[:1000])


b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\t\x07\x07\x02\x00\x00\x08\x06\x06\x0c\n\n\x02\x02\x02\n\n\n\x0e\x0e\x0e\x06\x06\x06\x03\x03\x03\x08\x08\x08\n\x08\x08\x03\x01\x01\x04\x00\x00\t\x04\x05\x0b\x05\x06\x06\x01\x02\x03\x01\x01\x08\x08\x08\r\r\r\x0b\x0b\x0b\x08\x06\x06\x06\x04\x04\x07\x05\x05\t\x07\x07\n\x08\x08\t\x07\x07\x06\x06\x06\x04\x06\x06\x04\x06\x06\x02\x07\x06\x03\x08\x07\x04\x08\t\x00\x04\x07\x06\n\x0f\t\x0c\x10\t\r\x0e\x0e\r\x0f\n\n\n\x08\x06\x05\x0b\t\x08\x0c\x08\x07\x0c\x08\x07\n\x08\x07\n\x08\x07\x08\t\x07\x07\x08\x06\x02\x04\x04\x00\x02\x02\x0b\x08\n\x05\x00\x01\x12\x04\x08\x0c\x03\x06\x06\n\x0b\x00\x0e\r\r" \x10\x1e\x1d\t\x08\n\x13\x0c\x0f\x1d\x16\x1b\x13\x12\x16\x00\x04\t\x01\x07\x0c\x13\x0c\x13\x0c\x06\x0b\t\x08\n\x05\n\t\x07\x0c\x0b\t\x0e\r\x0c\x11\x0f\x0f\x14\x12\x14\x17\x15\x16\x19\x17\x19\x1a\x18\x1f \x1e"""\x1b\x1b\x1b\x11\x10\x12\x0b\n\x0c\x0c\x08\r\r\t\x0e\x0e\r\x11\r\x0c\x10\x11\r\x12\x12\x0e\x